In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [3]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('chromium')

2023-01-25 16:10:23,885 INFO: ====== WebDriver manager ======
2023-01-25 16:10:24,153 INFO: Get LATEST chromedriver version for chromium None
2023-01-25 16:10:24,599 INFO: Driver [C:\Users\Chris\.wdm\drivers\chromedriver\win32\109.0.5414\chromedriver.exe] found in cache


**Scrape New Completed Games and Format Them**

In [4]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 5
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




https://www.nba.com/stats/teams/boxscores?DateFrom=01/20/23&DateTo=01/25/23
all pages loaded
<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 0 to 73
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            74 non-null     int64         
 1   GAME_DATE_EST   74 non-null     datetime64[ns]
 2   HOME_TEAM_WINS  74 non-null     int64         
 3   PTS             74 non-null     int64         
 4   FG_PCT          74 non-null     float64       
 5   FG3_PCT         74 non-null     float64       
 6   FT_PCT          74 non-null     float64       
 7   REB             74 non-null     int64         
 8   AST             74 non-null     int64         
 9   TEAM_ID         74 non-null     object        
 10  GAME_ID         74 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 6.9+ KB
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-24,0,110,48.9,20.8,76.0,45,19,1610612741,22200713,116,42.4,27.0,78.6,45,24,1610612754,2022
1,2023-01-24,0,95,43.2,31.3,77.8,44,21,1610612738,22200714,98,36.2,36.4,78.3,51,21,1610612748,2022
2,2023-01-24,0,103,46.0,36.4,68.8,41,29,1610612739,22200715,105,49.3,51.5,66.7,42,25,1610612752,2022
3,2023-01-24,1,133,60.7,50.0,80.0,47,25,1610612746,22200719,115,44.2,40.0,62.5,32,21,1610612747,2022
4,2023-01-24,1,99,48.8,33.3,76.9,34,27,1610612743,22200716,98,44.7,19.2,85.0,43,18,1610612740,2022
5,2023-01-24,0,97,40.6,24.0,72.2,39,19,1610612766,22200718,128,50.0,55.0,92.3,55,38,1610612756,2022
6,2023-01-24,1,127,50.6,36.7,78.0,41,28,1610612764,22200717,126,52.8,33.3,84.6,38,24,1610612742,2022
7,2023-01-23,0,100,40.8,33.3,80.0,31,28,1610612763,22200712,133,55.7,55.0,92.9,53,37,1610612758,2022
8,2023-01-23,0,102,44.7,12.5,70.6,42,22,1610612766,22200710,120,47.7,40.0,84.6,46,25,1610612762,2022
9,2023-01-23,0,114,45.1,38.1,72.7,41,19,1610612750,22200709,119,50.0,36.7,66.7,47,24,1610612745,2022


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(driver)


print(matchups)
print(game_ids)


[['1610612752', '1610612738'], ['1610612765', '1610612751'], ['1610612741', '1610612766'], ['1610612739', '1610612745'], ['1610612742', '1610612756'], ['1610612759', '1610612746']]
['22200730', '22200731', '22200732', '22200733', '22200734', '22200735']


**Close Webdriver**

In [6]:
driver.close() 

**Create Rows for Today's Games with Empty Stats**

In [7]:
# append today's matchups to the new games dataframe


df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': int(game_ids[i]),                       
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612738,22200730,0.0,0.0,0.0,0.0,0.0,0.0,1610612752,2022
1,2023-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612751,22200731,0.0,0.0,0.0,0.0,0.0,0.0,1610612765,2022
2,2023-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612766,22200732,0.0,0.0,0.0,0.0,0.0,0.0,1610612741,2022
3,2023-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612745,22200733,0.0,0.0,0.0,0.0,0.0,0.0,1610612739,2022
4,2023-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612756,22200734,0.0,0.0,0.0,0.0,0.0,0.0,1610612742,2022
5,2023-01-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612746,22200735,0.0,0.0,0.0,0.0,0.0,0.0,1610612759,2022


**Access Feature Store**

In [8]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [9]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [10]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2023-01-25 16:11:06,970 INFO: USE `nba_predictor_featurestore`
2023-01-25 16:11:07,354 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23467,2023-01-17,22200667,1610612743,1610612757,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
23468,2023-01-16,22200660,1610612744,1610612764,2022,127,49.500000,90.500000,40.906250,28,43,118,48.812500,87.000000,40.000000,31,36,1
23469,2023-01-17,22200665,1610612749,1610612761,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
23470,2023-01-16,22200661,1610612748,1610612737,2022,113,48.312500,95.000000,25.796875,30,41,121,59.187500,81.812500,48.093750,27,28,0


**Convert Feature Names back to original mixed case**

In [11]:
df_old = convert_feature_names(df_old)
df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23467,2023-01-17,22200667,1610612743,1610612757,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
23468,2023-01-16,22200660,1610612744,1610612764,2022,127,49.500000,90.500000,40.906250,28,43,118,48.812500,87.000000,40.000000,31,36,1
23469,2023-01-17,22200665,1610612749,1610612761,2022,0,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000,0,0,0
23470,2023-01-16,22200661,1610612748,1610612737,2022,113,48.312500,95.000000,25.796875,30,41,121,59.187500,81.812500,48.093750,27,28,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [12]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results

#df_old = df_old.set_index('GAME_ID')
#df_old.update(df_new.set_index('GAME_ID'))
#df_old = df_old.reset_index()  

# another approach is to simply drop the rows that were used for prediction yesterday
# which are games that have 0 points for home team
# and then append the new rows to the dataframe
df_old = df_old[df_old['PTS_home'] != 0]
df_old = pd.concat([df_old, df_new], ignore_index = True)

df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2009-03-15,20800989,1610612755,1610612748,2008,85,0.423096,0.666992,0.293945,24,40,77,0.412109,0.695801,0.262939,22,38,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23463,2023-01-20,22200685,1610612744,1610612739,2022,120,48.800000,68.400000,53.500000,25,45,114,50.000000,84.200000,42.400000,24,35,1
23464,2023-01-20,22200687,1610612746,1610612759,2022,131,63.000000,70.000000,53.600000,31,32,126,57.300000,71.400000,53.800000,34,34,1
23465,2023-01-20,22200684,1610612752,1610612737,2022,124,59.000000,76.500000,43.300000,27,34,139,56.000000,87.500000,38.200000,27,39,0
23466,2023-01-20,22200686,1610612748,1610612742,2022,90,47.800000,74.100000,20.000000,15,39,115,46.300000,82.100000,47.400000,21,37,0


**Add Today's Matchups for Feature Engineering**

In [13]:
df_combined = pd.concat([df_old, df_today], ignore_index = True)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08 00:00:00,21700374,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0
1,2013-03-01 00:00:00,21200874,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0
2,2005-11-30 00:00:00,20500210,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0
3,2009-03-15 00:00:00,20800989,1610612755,1610612748,2008,85.0,0.423096,0.666992,0.293945,24.0,40.0,77.0,0.412109,0.695801,0.262939,22.0,38.0,1.0
4,2004-11-17 00:00:00,20400109,1610612751,1610612760,2004,68.0,0.413086,0.889160,0.000000,9.0,37.0,79.0,0.406006,0.758789,0.333008,12.0,33.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23469,2023-01-25,22200731,1610612751,1610612765,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23470,2023-01-25,22200732,1610612766,1610612741,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23471,2023-01-25,22200733,1610612745,1610612739,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23472,2023-01-25,22200734,1610612756,1610612742,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


**Data Processing**

In [14]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,2017-12-08 00:00:00,21700374,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0,0,1.0
1,2013-03-01 00:00:00,21200874,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0,0,1.0
2,2005-11-30 00:00:00,20500210,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0,0,1.0
3,2009-03-15 00:00:00,20800989,1610612755,1610612748,2008,85.0,0.423096,0.666992,0.293945,24.0,40.0,77.0,0.412109,0.695801,0.262939,22.0,38.0,1.0,0,1.0
4,2004-11-17 00:00:00,20400109,1610612751,1610612760,2004,68.0,0.413086,0.889160,0.000000,9.0,37.0,79.0,0.406006,0.758789,0.333008,12.0,33.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23469,2023-01-25,22200731,1610612751,1610612765,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23470,2023-01-25,22200732,1610612766,1610612741,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23471,2023-01-25,22200733,1610612745,1610612739,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23472,2023-01-25,22200734,1610612756,1610612742,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0


**Feature Engineering**

In [15]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

**Insert New Data into Feature Group**

In [16]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/23474 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1b3fbd456d0>, None)

In [17]:
df_old.tail(20)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
23448,2023-01-22,22200702,1610612760,1610612743,2022,101,44.9,87.5,26.9,21,50,99,43.9,51.5,35.7,22,42,1
23449,2023-01-22,22200700,1610612752,1610612761,2022,116,48.8,69.2,43.9,30,47,125,45.6,80.0,44.1,21,45,0
23450,2023-01-22,22200701,1610612746,1610612742,2022,112,45.5,96.8,42.9,20,46,98,43.8,57.7,37.1,15,38,1
23451,2023-01-22,22200704,1610612751,1610612744,2022,120,54.4,64.5,51.9,30,42,116,45.6,77.8,40.6,28,42,1
23452,2023-01-21,22200694,1610612766,1610612737,2022,122,50.6,86.4,38.2,28,36,118,54.2,85.0,44.0,24,40,1
23453,2023-01-21,22200696,1610612745,1610612750,2022,104,48.1,57.6,37.5,24,56,113,45.8,83.3,41.5,20,30,0
23454,2023-01-21,22200697,1610612754,1610612756,2022,107,37.5,84.6,35.1,22,48,112,41.8,68.6,38.7,27,55,0
23455,2023-01-21,22200692,1610612738,1610612761,2022,106,44.7,93.3,45.7,25,41,104,53.5,71.4,36.8,24,41,1
23456,2023-01-21,22200698,1610612755,1610612758,2022,129,52.7,82.6,42.9,27,46,127,54.8,75.9,39.4,28,29,1
23457,2023-01-21,22200695,1610612749,1610612739,2022,102,45.3,90.9,35.9,24,36,114,55.8,64.7,31.8,33,41,0


In [18]:
df_combined[df_combined['GAME_ID'] > 22200640]

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
23407,2023-01-14,22200641,1610612749,1610612748,2022,95,40.50000,66.6875,39.59375,26,...,1.879687,0.347917,4.333333,1.857143,1.0,0.333333,0.333333,6.714286,10.2,9.200000
23408,2023-01-14,22200646,1610612755,1610612762,2022,118,44.90625,80.8125,43.59375,23,...,4.151563,2.839583,8.333333,2.714286,1.4,2.733333,-7.000000,-2.857143,-5.2,-1.533333
23409,2023-01-14,22200645,1610612739,1610612750,2022,102,48.68750,87.0000,28.59375,24,...,-1.815625,-1.963542,2.000000,-0.142857,0.9,-0.066667,1.000000,4.000000,2.5,1.666667
23410,2023-01-14,22200642,1610612738,1610612766,2022,122,47.50000,90.0000,35.81250,35,...,0.703125,-0.718750,-4.333333,-0.571429,-0.1,-0.933333,7.666667,1.714286,0.1,2.066667
23411,2023-01-14,22200643,1610612763,1610612754,2022,130,55.18750,78.6250,37.50000,32,...,0.153125,-2.714583,-1.666667,-1.000000,-1.2,-0.866667,4.333333,8.285714,7.8,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23469,2023-01-25,22200735,1610612746,1610612759,2022,0,0.00000,0.0000,0.00000,0,...,1.348438,2.517708,-5.000000,-5.857143,-5.4,-3.733333,3.666667,3.428571,1.4,0.400000
23470,2023-01-25,22200734,1610612756,1610612742,2022,0,0.00000,0.0000,0.00000,0,...,4.439062,5.519792,10.000000,6.571429,5.5,4.333333,12.333333,9.714286,7.6,5.400000
23471,2023-01-25,22200733,1610612745,1610612739,2022,0,0.00000,0.0000,0.00000,0,...,-0.334375,-1.230208,-3.333333,-0.142857,-1.0,-1.000000,11.333333,4.571429,4.5,5.000000
23472,2023-01-25,22200732,1610612766,1610612741,2022,0,0.00000,0.0000,0.00000,0,...,-3.171875,-5.246875,-2.333333,1.857143,1.9,2.933333,-4.333333,-2.857143,-1.1,1.266667
